In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tifffile import imsave, imread
import os

os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP/gap')
from BinomDatasetv4 import BinomDataset 
from GAP_UNET_ResBlockv3 import UN
from tasks import inpainting

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
import torch.utils.data as dt

if not torch.cuda.is_available():
    raise ValueError("GPU not found, code will run on CPU and can be extremely slow!")
else:
    device = torch.device("cuda:0")

print(f'Device in use: {device}')

Device in use: cuda:0


In [2]:
data = np.concatenate((imread('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/convallaria/trainingDataGT.tif'), imread('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/convallaria/testDataGT.tif')))
# plt.imshow(data[0])
print(data.shape)

(2048, 512, 512)


In [3]:
name = 'm40tom5-256x256-inpainting-trial'
minpsnr = -40
maxpsnr = -5
CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/'

In [4]:
maxepochs = 2

# masks = inpainting()
# mask = torch.from_numpy(masks.generate_mask())

dataset = BinomDataset(data = data[:round(data.shape[0]*0.9)], windowSize= 256, minPSNR=minpsnr, maxPSNR=maxpsnr)
val_dataset = BinomDataset(data = data[round(data.shape[0]*0.9):], windowSize=256, minPSNR=minpsnr, maxPSNR=maxpsnr)

train_loader = dt.DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True, pin_memory=False, num_workers=1) ## Changing the batch size from 32 to 16 to fit inside the gpu
val_loader = dt.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False,  pin_memory=False, num_workers=1)

trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, name), gradient_clip_val=0.5,
                     accelerator= 'gpu',
                     max_epochs=maxepochs, 
                     callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="val_loss", every_n_epochs = 5, filename= name, dirpath= CHECKPOINT_PATH),
                                LearningRateMonitor("epoch"),
                                EarlyStopping('val_loss', patience=2000)])

model = UN(channels = 1, levels=1, depth=2,
           start_filts=28, up_mode = 'transpose').to(device)